In [ ]:
from extraction import extract_articles_from_google
from extraction import extract_data_from_page
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By

In [11]:
pages_file_name = "ex_ante_articles.json"
content_file_name = "ex_ante_articles_with_content.json"

In [ ]:
extract_articles_from_google(
	website="ex-ante.cl",
	newscast="Ex-Ante",
    pages=2,
    output=pages_file_name,
    keywords="Estallido Social",
    )

In [14]:
def get_content(body: WebElement):
	words_to_avoid = [
		"También puede leer.",
		"Lea también",
		"Para seguir leyendo columnas de Ex-Ante, clic aquí.",
		"(Lea aquí la encuesta completa)"
	]
	content = ""
	for element in body.find_element(By.CLASS_NAME, "contenido-noticia").find_elements(By.TAG_NAME, "p"):
		if any(phrase in element.text for phrase in words_to_avoid):
			continue
		content += element.text.strip() + "\n"
	return content

def get_author(body: WebElement):
	try:
		return body.find_element(By.CLASS_NAME, "autor").text.strip()
	except Exception:
		return "The website does not provide an author"

def get_description(body: WebElement):
	return "not found"

extract_data_from_page(
	input_file=pages_file_name, 
	output_file=content_file_name,
	get_author=get_author,
	get_description=get_description,
	get_content=get_content,
	limit_of_pages=5,
	wait=60,
)

[INFO] Input file: ex_ante_articles.json
[INFO] Output file: ex_ante_articles_with_content.json
[INFO] Limit of pages to process: 5

[INFO] Loading extracted pages from JSON file...
[INFO] Starting extraction process...

[INFO] Checking page: 0 | Link: https://www.ex-ante.cl/cronica-de-una-escualida-y-violenta-protesta-en-plaza-baquedano-a-cuatro-anos-del-estallido/
[INFO] Storing information in a JSON file...

[INFO] Checking page: 1 | Link: https://www.ex-ante.cl/estallido-social-la-necesidad-de-separar-causas-y-consecuencias-por-cristian-valdivieso/
[ERROR] An error occurred while processing page 1: Message: session not created: cannot connect to chrome at 127.0.0.1:57773
from chrome not reachable; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
	GetHandleVerifier [0x0x125d2a3+66419]
	GetHandleVerifier [0x0x125d2e4+66484]
	(No symbol) [0x0x1034a20]
	(No symbol) [0x0x102866b]

In [ ]:

months = {
    "ene": "Jan", "feb": "Feb", "mar": "Mar", "abr": "Apr",
    "may": "May", "jun": "Jun", "jul": "Jul", "ago": "Aug",
    "sept": "Sep", "oct": "Oct", "nov": "Nov", "dic": "Dec"
}

locale.setlocale(locale.LC_TIME, 'C')

json_pages_info = {"pages": []}


options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = uc.Chrome(options=options)

pages = 10

for page in range(0, pages):

	url = f'https://www.google.com/search?q=%22Estallido+social%22+site%3Awww.ex-ante.cl&tbs=cdr:1,cd_min:11/15/2019,cd_max:12/17/2023&start={page * 10}'

	driver.get(url)

	time.sleep(2)

	articles_section = driver.find_element(By.CLASS_NAME, "dURPMd")
	articles = articles_section.find_elements(By.CLASS_NAME, "MjjYud")

	for article in articles:
		print("revisando artículo...")
		try: 
			originalDate = article.find_element(By.CLASS_NAME, "YrbPuc").find_element(By.TAG_NAME, "span").text
			for es, en in months.items():
				if es in originalDate:
						originalDate = originalDate.replace(es, en)
						break
				
			date_epoch = int(time.mktime(time.strptime(originalDate, "%d %b %Y")))

			if (date_epoch < 1573786800 or date_epoch > 1702782000):
				print("La noticia no corresponde a la fecha solicitada")
				continue

			try:
				title = article.find_element(By.TAG_NAME, "h3").text
				description = article.find_element(By.CLASS_NAME, "kb0PBd ").find_elements(By.TAG_NAME, "span")[1].text
				link = article.find_element(By.TAG_NAME, "a").get_attribute("href")

				link_info = {
					"newscast" : "El Mostrador",
					"title": title,
					"description": description,
					"category": "The site does not provide a category",
					"date": originalDate,
					"image_link": "not found initially",
					"author": "not found initially",
					"link": link,
				}

				print("información incluida!")
				json_pages_info["pages"].append(link_info)

			except Exception as e:
				print(f"Error al extraer información del artículo: {e}")
				continue

		except Exception as e:
			print(f"Error al procesar el artículo: {e}")
			continue

print("Almacenando información en el archivo JSON...")
with open(f"../archive/temp/el_mostrador_pages.json", 'w', encoding='utf-8') as file:
	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)


with open(f"output.html", "w", encoding="utf-8") as f:
	f.write(driver.page_source)

driver.quit()

## Extraction of content

In [ ]:

extracted_pages = {"pages": []}
extracted_pages_with_content = {"pages": []}
total_pages = 0
pages_succeeded = 0
pages_failed = 0
error_occurred = False
driver = None
ad_message = "Suscríbase aquí a la newsletter de EL PAÍS América y reciba todas las claves informativas de la actualidad de la región."
limit_of_pages = "all"
start_page = 10

content = ""
author = ""

print("[INFO] Loading extracted pages from JSON file...")
with open("../archive/pages_extracted/ex_ante/ex_ante_pages.json", 'r', encoding='utf-8') as file:
		extracted_pages = json.load(file)
		print(f"[INFO] {len(extracted_pages['pages'])} pages loaded.")
		if limit_of_pages == "all":
			limit_of_pages = len(extracted_pages["pages"])


if start_page >= len(extracted_pages["pages"]):
		print(f"[ERROR] start_page is out of range. The maximum page index is {len(extracted_pages['pages']) - 1}.")
		raise SystemExit

def extract_from_DOM():
		print("[INFO] Trying to extract content from DOM...")
		content = ""
		author = ""

		content_container = driver.find_element(By.CLASS_NAME, "contenido-noticia")
		paragraph = content_container.find_elements(By.TAG_NAME, "p") if content_container else None
		content = ""
		for p in paragraph:
			p = p.text.strip()
			if "También puede leer." in p or "Lea también:" in p or "Para seguir leyendo columnas de Ex-Ante, clic aquí." in p:
					break
			content += p + "\n"

		try:
				author = driver.find_element(By.CLASS_NAME, "autor").text.strip()
		except:
				author = "not found"

		print("[INFO] The content has been extracted. Method: DOM", flush=True)
		if len(content) < 100:
				print("[WARNING] The extracted content is too short.", flush=True)
				print(f"[WARNING] Content: {content}", flush=True)
		return content, author

data_extracted = True
re = False

print("[INFO] Starting extraction process...")
for page_index, page in enumerate(extracted_pages["pages"]):

		if "content" not in page.keys():
				page["content"] = "The content has not been extracted yet"

		if page_index < start_page:
				print("[INFO] Skipping page: ", page_index)
				continue
		if limit_of_pages == total_pages:
				print("[INFO] Limit of pages reached.")
				break
		if page["content"] != "The content has not been extracted yet":
				print(f"[INFO] Page {page_index} already has content. Skipping...")
				extracted_pages_with_content["pages"].append(page)
				continue

		total_pages += 1

		print(f"\n[INFO] Checking page: {page_index} | Link: {page['link']}" )
		url = page['link']
		first = True

		while (re or first):
			first = False
			
			try: 
				time.sleep(1)

				if (not re): 
					options = uc.ChromeOptions()
					options.add_argument("--no-sandbox")
					options.add_argument("--disable-blink-features=AutomationControlled")

					driver = uc.Chrome(options=options)

					driver.get(url)

				re = False
				time.sleep(random.uniform(6, 12))

				try: 
					content, author = extract_from_DOM()
					if author == "not found":
							try:
									author = driver.find_element(By.CLASS_NAME, "a_md_a").text.strip()
							except:
									author = "not found"

					# Clean up ad message
					content = content.replace(ad_message, "").strip()
				except Exception as e:
					print(f"[ERROR] A error occurred while extracting content", flush=True)
					raise

			except Exception as e:
				print(f"[ERROR] A error occurred while processing the page: {page_index}", flush=True)
				print(f"[ERROR] Site: {url}", flush=True)
				print(e, flush=True)
				response = input("Retry? y/n")
				if response.lower() == 'y':
					print("[INFO] Retry...")
					re = True
				else: 
					print("[INFO] Cancelling...")
					error_occurred = True
					re = False
			
			if (not re):
				driver.quit()

		if error_occurred:
			content = "A error occurred while extracting content"
			author = "A error occurred while extracting content"
			pages_failed += 1
			error_occurred = False
		else:
			pages_succeeded += 1

		page["content"] = content
		page["author"] = author
		extracted_pages_with_content["pages"].append(page)

		print("[INFO] Storing information in a JSON file...")
		with open(f"../archive/temp/pages_with_content/ex_ante_pages_with_content.json", 'w', encoding='utf-8') as file:
			json.dump(extracted_pages_with_content, file, ensure_ascii=False, indent=4)

		driver.quit()

		time.sleep(random.uniform(6, 12))

# ----------------- Stats -----------------
print("\n\n")
print("----- STATS -----")

try:
	print(f"Amount of pages processed: {total_pages}")
	print(f"% Pages succeeded [{pages_succeeded}]: {round(100 * pages_succeeded/total_pages, 2)}")
	print(f"% Pages failed [{pages_failed}]: {round(100 * pages_failed/total_pages, 2)}")
except: 
	print("A error occurred while processing the stats")